In [6]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ingma\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [7]:
articles = pd.read_json("data/articles.json.xz")
google_qa = pd.read_json("data/google_qa.json.xz")
jokes = pd.read_json("data/jokes.json.xz")

In [17]:
datas = {}
stop_words = stopwords.words('english')
lemmatizer = WordNetLemmatizer()
stop_words.extend(["...", "'s", "wo", "n't", "'m", "ca", "'ll", "'re", "'ve", "'d"])

all_lem_data = []
for name, data, n in zip(['articles', 'google_qa', 'jokes'], [articles, google_qa, jokes], [50000, 50000, 100000]):
    print(name, end="\n\n\n")
    lemmatized_data = []
    for i, row in data.iterrows():
        if i == n:
            break
        if i % 10000 == 0:
            print(i)
        text = row['text']
        words = word_tokenize(text)
        lemmas = []

        for word in words:
            lemma = lemmatizer.lemmatize(word)
            if len(lemma) > 1 and not lemma in stop_words:
                lemmas.append(lemma.lower())
        lemmatized_row = {'id': f"{name}_{i}",'text': " ".join(lemmas), 'joke': name == 'jokes'}
        lemmatized_data.append(lemmatized_row)
        all_lem_data.append(lemmatized_row)
    datas[name] = pd.DataFrame(lemmatized_data)


articles


0
10000
20000
30000
40000
google_qa


0
10000
20000
30000
40000
jokes


0
10000
20000
30000
40000
50000
60000
70000
80000
90000


In [18]:
for key in datas:
    data = datas[key]
    data.to_json(path_or_buf=f"{key}_lemmas.json")

In [19]:
all_lem_df = pd.DataFrame(all_lem_data)
all_lem_df.to_json(path_or_buf="all_lemmas.json")
